# Crash Map Video

Make a video showing crashes over time on a map

Data source: https://www.denvergov.org/opendata/dataset/city-and-county-of-denver-traffic-accidents

Based on: https://medium.com/udacity/creating-map-animations-with-python-97e24040f17b

In [1]:
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM, Stamen
import pandas as pd
from tqdm.notebook import tqdm
import geopy
import geopy.distance

from crash_data_analysis import CrashDataAnalysis
cda = CrashDataAnalysis()

In [2]:
# East Speer

# frame_directory = 'video_speer'
# paste_center = [39.726591, -104.986417]
# miles_vertical = 0.9
# zoom_level = 16

In [3]:
# Westwood

# frame_directory = 'video_kentucky'
# paste_center = [39.70039857002305, -105.03918676552246]
# miles_vertical = 1.7
# zoom_level = 15

In [4]:
# Downtown - Broadway / 20th St / California

# frame_directory = 'video_downtown'
# paste_center = [39.74891627011233, -104.98736403434121]
# miles_vertical = 1.2
# zoom_level = 16

In [5]:
# CDOT HQ

frame_directory = 'video_federal'
paste_center = [39.73802448372717, -105.02517462078]
miles_vertical = 0.5
zoom_level = 17

In [6]:
date_first = datetime(2022,1,1)
date_last = datetime(2022,12,31) # df.crash_timestamp.max()

In [7]:
if not os.path.exists(frame_directory):
    os.makedirs(frame_directory)

In [8]:
d_vertical = geopy.distance.distance(miles=(miles_vertical)/2)
d_horizontal = geopy.distance.distance(miles=(miles_vertical * 1.25)/2)

center = geopy.Point(latitude=paste_center[0], longitude=paste_center[1])

bounds = {}
bounds['left'] = d_horizontal.destination(point=center, bearing=270).longitude
bounds['top'] = d_vertical.destination(point=center, bearing=0).latitude
bounds['right'] = d_horizontal.destination(point=center, bearing=90).longitude
bounds['bottom'] = d_vertical.destination(point=center, bearing=180).latitude

In [9]:
square_miles = (miles_vertical * 1.25) * miles_vertical
square_miles

0.3125

In [10]:
query = f"""
select 
*
, reported_date at time zone 'America/Denver' as reported_date_local

from crashes

where {bounds['left']} < geo_lon
and geo_lon < {bounds['right']}
and {bounds['bottom']} < geo_lat
and geo_lat < {bounds['top']}

and '{date_first}' <= reported_date at time zone 'America/Denver'
and reported_date at time zone 'America/Denver' <= '{date_last}'

order by reported_date
"""

df_in_bounds = pd.read_sql(query, cda.conn)

In [11]:
print('Stats about crash database: ')
print(f'Number of crashes: {len(df_in_bounds):,}')
print(f'Unique IDs: {df_in_bounds.incident_id.nunique():,}')
print(f'Min date: {df_in_bounds.reported_date_local.min()}')
print(f'Max date: {df_in_bounds.reported_date_local.max()}')

Stats about crash database: 
Number of crashes: 69
Unique IDs: 69
Min date: 2022-01-03 16:01:00
Max date: 2022-12-27 21:53:00.000001


In [12]:
stats_by_date = df_in_bounds.groupby('crash_date').agg(
    crashes=('incident_id', 'size')
    , fatalities=('fatality', 'sum')
    , serious_injuries=('sbi', 'sum')
    , pedestrians=('pedestrian_ind', 'sum')
    , bicycles=('bicycle_ind', 'sum')
)

# stats_by_date.to_clipboard()

stats_by_date.sum()

crashes             69.0
fatalities           1.0
serious_injuries     3.0
pedestrians          4.0
bicycles             1.0
dtype: float64

In [13]:
df_in_bounds['marker_shape'] = 'o' # circle
df_in_bounds.loc[df_in_bounds.pedestrian_ind > 0, 'marker_shape'] = 'D' # diamond
df_in_bounds.loc[df_in_bounds.bicycle_ind > 0, 'marker_shape'] = 's' # square

# https://colorbrewer2.org/#type=diverging&scheme=RdYlBu&n=4
# https://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=3
colors = {}
colors['crashes'] = '#1b9e77' # green
colors['serious_injuries'] = '#d95f02' # orange
colors['fatalities'] = '#7570b3' # purple

df_in_bounds['marker_color'] = colors['crashes']
df_in_bounds.loc[df_in_bounds.sbi, 'marker_color'] = colors['serious_injuries']
df_in_bounds.loc[df_in_bounds.fatality, 'marker_color'] = colors['fatalities']

In [14]:
# Save the base map
imagery = Stamen(style='toner-lite', cache=True)

In [15]:
fig = plt.figure(figsize=[16,9])
fig.subplots_adjust(left=0, bottom=0.05, right=0.7, top=0.95)

ax = plt.axes(projection=imagery.crs)
ax.add_image(imagery, zoom_level)
ax.set_extent((bounds['left'], bounds['right'], bounds['bottom'], bounds['top']))

# plt.savefig(f'{frame_directory}/base_map.png', facecolor='white')
plt.close()

In [16]:
def save_one_frame(df, map_date, bounds, zoom_level, frame_filename, colors):
    """
    Save one frame of the video with crashes from that day and previous days
    
    df : DataFrame containing crashes (already limited to the bounds). If None, only base map is saved
    map_date : date for the crashes that will be plotted
    bounds : dict containing left (lon), right (lon), top (lat), and bottom (lat) bounds of the map
    frame_filename : the filename where to save the frame PNG
    colors : dict containing colors for 'crashes', 'serious_injuries', and 'fatalities'
    """

    fig = plt.figure(figsize=[16,9])

    fig.subplots_adjust(left=0, bottom=0.05, right=0.7, top=0.95)

    ax = plt.axes(projection=imagery.crs)
    ax.set_extent((bounds['left'], bounds['right'], bounds['bottom'], bounds['top']))
    ax.add_image(imagery, zoom_level)
    
    for days_back in range(27,-1,-1):
        
        rows_to_plot = df.crash_date == (map_date - timedelta(days=days_back)).strftime('%Y-%m-%d')

        if rows_to_plot.sum() > 0:
            for m in df[rows_to_plot]['marker_shape'].unique():
                for c in df[rows_to_plot]['marker_color'].unique():
                    
                    # Make the crashes fade out sooner than the injuries and fatalities
                    if c == colors['crashes']:
                        fill_style='none'
                        if days_back >= 14:
                            alpha_value = 0
                        elif days_back >= 7:
                            alpha_value = 0.8 - (0.1 * (days_back - 6))
                        else:
                            alpha_value = 0.8

                    else: # fatalities and serious injuries
                        fill_style = 'full'
                        if days_back >= 21:
                            alpha_value = 0.8 - (0.1 * (days_back - 20))
                        else:
                            alpha_value = 0.8


                    ax.plot(
                        df.loc[
                            (rows_to_plot) & (df['marker_shape'] == m) & (df['marker_color'] == c)
                            , 'geo_lon'
                            ].to_numpy()
                        , df.loc[
                            (rows_to_plot) & (df['marker_shape'] == m) & (df['marker_color'] == c)
                            , 'geo_lat'
                            ].to_numpy()
                        , transform=ccrs.Geodetic()
                        , marker=m
                        , color=c
                        , linestyle=''
                        , markersize=(5*(days_back+1)) + 10
                        , fillstyle=fill_style
                        , markeredgewidth=6
                        , alpha=alpha_value
                    )


    # Do counts of crashes up to map_date
    counts = df[df.reported_date_local <= map_date].groupby('crash_date').agg(
        crashes=('incident_id', 'size')
        , fatalities=('fatality', 'sum')
        , serious_injuries=('sbi', 'sum')
        , pedestrians=('pedestrian_ind', 'sum')
        , bicycles=('bicycle_ind', 'sum')
    ).sum()    
    
    legend = [
        ('Date: {}'.format(map_date.strftime('%Y-%m-%d')), 'black', 0.91)
        , ('Car crashes:{: 6,.0f}'.format(counts['crashes']), colors['crashes'], 0.78)
        , ('Crashes with a       \nserious injury:{: 6,.0f}'.format(
            counts['serious_injuries']), colors['serious_injuries'], 0.6175)
        , ('Crashes with a       \nfatality:{: 6,.0f}'.format(counts['fatalities']), colors['fatalities'], 0.455)
        , (u"\n\u25C7 Pedestrian       \ninvolved:{: 6,.0f}".format(counts['pedestrians']), 'black', 0.2925)
        , (u"\n\u25FB Bicycle       \ninvolved:{: 6,.0f}".format(counts['bicycles']), 'black', 0.13)
    ]
    
    for idx, line in enumerate(legend):
            
        plt.gcf().text(
            x=0.97
            , y=line[2]
            , s=line[0]
            , fontsize=26
            , horizontalalignment='right'
            , verticalalignment='bottom'
#             , family='monospace'
            , font='Menlo'
            , color=line[1]
        )

    plt.gcf().text(
        x=0.97
        , y=0.05
        , s='Visualization by Devin Brady'
        , fontsize=18
        , horizontalalignment='right'
#         , family='monospace'
        , font='Menlo'
        , color='black'
    )

    plt.savefig(frame_filename, facecolor='white')
    plt.close()
    
    return legend

In [17]:
video_date_range = pd.date_range(date_first,date_last,freq='d')

for counter, d in enumerate(tqdm(video_date_range, 'Saving map frames')):

    # only save the last image
#     if counter < 364:
#         continue
    
    legend = save_one_frame(df_in_bounds, d, bounds, zoom_level, f'{frame_directory}/frame_{counter:04d}.png', colors)



Saving map frames:   0%|          | 0/365 [00:00<?, ?it/s]

In [18]:
# Save the initial map
_ = save_one_frame(
    pd.DataFrame(columns=df_in_bounds.columns)
    , video_date_range[0]
    , bounds
    , zoom_level
    , f'{frame_directory}/initial_map.png'
    , colors
)

In [19]:
# df_in_bounds.info()

## Terminal command 

In [23]:
print(
    f'ffmpeg -framerate 12 -i {frame_directory}/frame_%4d.png -c:v h264 -r 30 '
    + f'-s 1920x1080 {frame_directory}/crashes.mp4'
)

ffmpeg -framerate 12 -i video_federal/frame_%4d.png -c:v h264 -r 30 -s 1920x1080 video_federal/crashes.mp4


In [20]:
# import numpy as np
# np.linspace(0.75, 0.1, num=5)

In [21]:
# import matplotlib
# matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf') #[:10]

In [22]:
for leg in legend:
    print(leg[0])

Date: 2022-12-31
Car crashes:    69
Crashes with a       
serious injury:     3
Crashes with a       
fatality:     1

◇ Pedestrian       
involved:     4

◻ Bicycle       
involved:     1
